## 产生待分析日志的bochs代码
[所用bochs代码](https://gitcode.net/crk/bochs/-/commit/1871e377d237e70380c7256ee569c7f610b21212)

## 环境准备


jupyter环境安装
```shell
cd /crk/oslab/
wget https://mirrors.tuna.tsinghua.edu.cn/anaconda/miniconda/Miniconda3-py37_23.1.0-1-Linux-x86_64.sh
#...
```

```shell
python -m pip install --upgrade pip
pip config set global.index-url https://pypi.tuna.tsinghua.edu.cn/simple
```

```shell
pip install pandas
```

```shell
jupyter notebook --ip='0.0.0.0' --no-browser
```

打开[jupyter web client]( http://192.168.1.6:8888/)

## bochs运行linux-0.11, 产生日志文件  bochsout.txt

具体步骤参照:  [bochs运行linux-0.11, 产生日志文件  bochsout.txt](https://gitcode.net/crk/oslab/-/blob/master/README.MD)
```shell
#其中有一步:
/crk/bochs/bochs/bochs  -f /crk/oslab/bochs/bochsrc.bxrcsd
#产生了日志 /crk/oslab/bochsout.txt
```



## 日志bochsout.txt处理为bochsout.csv

### 记录当前时刻 并 查看日志

In [1]:
! echo -n "当前时刻:" && date ;    ls -lht bochsout* ; grep 日志 bochsout.txt | head -n 10 

当前时刻:2023年 11月 05日 星期日 21:57:28 CST
-rw-rw-r-- 1 z z 27M 11月  5 21:52 bochsout.csv
-rw-rw-r-- 1 z z 27M 11月  5 21:39 bochsout.txt
00000319494i[CPU0  ] 记录日志;模拟指令LGDT;LGDT_Ms;此行在区;此行内容;cpu_mode:0,_CS_selector_value:0xf000,_CS_selector_index:0x1e00,EIP:0x9a6f,eaddr:0x9af1,base_32:0xf9af7,limit_16:0x30;
00035768438i[CPU0  ] 记录日志;模拟指令LGDT;LGDT_Ms;此行在区;此行内容;cpu_mode:0,_CS_selector_value:0x9020,_CS_selector_index:0x4,EIP:0xa7,eaddr:0x132,base_32:0x90314,limit_16:0x800;
00035769801i[CPU0  ] 记录日志;模拟指令LGDT;LGDT_Ms;此行在区;此行内容;cpu_mode:2,_CS_selector_value:0x8,_CS_selector_index:0x1,EIP:0xa8,eaddr:0x54b0,base_32:0x5cb8,limit_16:0x7ff;
00035822742i[CPU0  ] 记录日志;模拟指令LTR;LTR_Ew;此行在区;此行内容;cpu_mode:2,_CS_selector_value:0x8,_CS_selector_index:0x1,EIP:0x76d0,raw_selector:0x20,selector:0x40020,descriptor:0x1;
00035822742i[CPU0  ] 记录日志;模拟指令LLDT;LLDT_Ew;此行在区;此行内容;cpu_mode:2,_CS_selector_value:0x8,_CS_selector_index:0x1,EIP:0x76d5,raw_selector:0x28,selector:0x50028,descriptor:0x1;
00035959550i[CPU0  ] 记录日志;未分

## 一LGDT日志行对应到了linux-0.11/boot/setup.s中的LGDT指令了

上面grep输出的第2行：

```00035768438i[CPU0  ] 记录日志;模拟指令LGDT;LGDT_Ms;此行在区;此行内容;cpu_mode:0,_CS_selector_value:0x9020,_CS_selector_index:0x4,EIP:0xa7,eaddr:0x132,base_32:0x90314,limit_16:0x800;```

此LGDT来自 :
[linux-0.11/boot/setup.s](https://gitcode.net/crk/hit-oslab-linux/-/blob/5fa782fa1be63d0fbc11dc697134c80a7b05e200/oslab/linux-0.11/boot/setup.s) 中的第133行
```asm
	lidt	idt_48		! load idt with 0,0 ;第132行
	lgdt	gdt_48		! load gdt with whatever appropriate ; 第133行

	call	empty_8042; 第137行
```

理由是
[linux-0.11/boot/setup.list](https://gitcode.net/crk/hit-oslab-linux/-/blob/5fa782fa1be63d0fbc11dc697134c80a7b05e200/oslab/linux-0.11/boot/setup.list) 中的
```
00133 00A2         0F0116       0132>           	lgdt	gdt_48		! load gdt with whatever appropriate
00137 00A7           E8         005F            	call	empty_8042
```
```00A7``` 就是出现在以上日志行中的 ```EIP:0xa7```

### 日志文件bochsout.txt转换为bochsout.csv

In [2]:
!grep 日志 bochsout.txt >bochsout.csv && ls -lh bochsout.* && ls -l bochsout.*

-rw-rw-r-- 1 z z 27M 11月  5 21:57 bochsout.csv
-rw-rw-r-- 1 z z 27M 11月  5 21:39 bochsout.txt
-rw-rw-r-- 1 z z 28255937 11月  5 21:57 bochsout.csv
-rw-rw-r-- 1 z z 28279670 11月  5 21:39 bochsout.txt


###  bochsout.csv添加列名

In [3]:
!  sed -i '1i\忽略头;函数分类;函数名;此行在区;此行内容;名值们;空尾' bochsout.csv

###  查看bochsout.csv

In [4]:
!head -n 8 bochsout.csv

忽略头;函数分类;函数名;此行在区;此行内容;名值们;空尾
00000319494i[CPU0  ] 记录日志;模拟指令LGDT;LGDT_Ms;此行在区;此行内容;cpu_mode:0,_CS_selector_value:0xf000,_CS_selector_index:0x1e00,EIP:0x9a6f,eaddr:0x9af1,base_32:0xf9af7,limit_16:0x30;
00035768438i[CPU0  ] 记录日志;模拟指令LGDT;LGDT_Ms;此行在区;此行内容;cpu_mode:0,_CS_selector_value:0x9020,_CS_selector_index:0x4,EIP:0xa7,eaddr:0x132,base_32:0x90314,limit_16:0x800;
00035769801i[CPU0  ] 记录日志;模拟指令LGDT;LGDT_Ms;此行在区;此行内容;cpu_mode:2,_CS_selector_value:0x8,_CS_selector_index:0x1,EIP:0xa8,eaddr:0x54b0,base_32:0x5cb8,limit_16:0x7ff;
00035822742i[CPU0  ] 记录日志;模拟指令LTR;LTR_Ew;此行在区;此行内容;cpu_mode:2,_CS_selector_value:0x8,_CS_selector_index:0x1,EIP:0x76d0,raw_selector:0x20,selector:0x40020,descriptor:0x1;
00035822742i[CPU0  ] 记录日志;模拟指令LLDT;LLDT_Ew;此行在区;此行内容;cpu_mode:2,_CS_selector_value:0x8,_CS_selector_index:0x1,EIP:0x76d5,raw_selector:0x28,selector:0x50028,descriptor:0x1;
00035959550i[CPU0  ] 记录日志;未分类;protected_mode_int;此行在区gate_descriptor.type=14;此行内容load_cs、load_ss、EIP=gate_dest_offset;gate_descr

##  jupyter单个cell允许多次输出

In [5]:
%config InteractiveShell.ast_node_interactivity = 'all'

##  导包

In [6]:
import pandas
# pandas.options.display.max_columns = None
# pandas.options.display.max_rows = None

pandas.options.display.max_columns = 1000
pandas.options.display.max_rows = 1000

## 读取csv

In [7]:
df0=pandas.read_csv(filepath_or_buffer="bochsout.csv",sep=';' , quotechar="'", 
                    usecols=["函数分类", "函数名", "此行在区", "此行内容", "名值们"])
df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74162 entries, 0 to 74161
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   函数分类    74162 non-null  object
 1   函数名     74162 non-null  object
 2   此行在区    74162 non-null  object
 3   此行内容    74162 non-null  object
 4   名值们     74162 non-null  object
dtypes: object(5)
memory usage: 2.8+ MB


In [8]:
df0

,函数分类,函数名,此行在区,此行内容,名值们
0,模拟指令LGDT,LGDT_Ms,此行在区,此行内容,"cpu_mode:0,_CS_selector_value:0xf000,_CS_selec..."
1,模拟指令LGDT,LGDT_Ms,此行在区,此行内容,"cpu_mode:0,_CS_selector_value:0x9020,_CS_selec..."
2,模拟指令LGDT,LGDT_Ms,此行在区,此行内容,"cpu_mode:2,_CS_selector_value:0x8,_CS_selector..."
3,模拟指令LTR,LTR_Ew,此行在区,此行内容,"cpu_mode:2,_CS_selector_value:0x8,_CS_selector..."
4,模拟指令LLDT,LLDT_Ew,此行在区,此行内容,"cpu_mode:2,_CS_selector_value:0x8,_CS_selector..."
...,...,...,...,...,...
74157,未分类,protected_mode_int,此行在区gate_descriptor.type=15,此行内容load_cs、load_ss、EIP=gate_dest_offset,"gate_descriptor.type:15,EIPorigin:0x690c,vecto..."
74158,未分类,protected_mode_int,此行在区gate_descriptor.type=15,此行内容load_cs、load_ss、EIP=gate_dest_offset,"gate_descriptor.type:15,EIPorigin:0x690c,vecto..."
74159,未分类,protected_mode_int,此行在区gate_descriptor.type=14,此行内容load_cs、load_ss、EIP=gate_dest_offset,"gate_descriptor.type:14,EIPorigin:0x6c04,vecto..."
74160,未分类,protected_mode_int,此行在区gate_descriptor.type=15,此行内容load_cs、load_ss、EIP=gate_dest_offset,"gate_descriptor.type:15,EIPorigin:0x690c,vecto..."


## 字符串 列 名值们 转为 Dict列 名值字典

### 工具

In [9]:
def 名值们toDict(名值们:str,row):
#     名值们='eaddr:0x9af1,base_32:0xf9af7,limit_16:0x30'
    try:
        Ls1=[名值k.split(':') for 名值k in 名值们.split(',')]
        return dict(Ls1)
    except Exception as e:
        print(f"行:{row}")
        raise e

### 转换

In [10]:
df0['名值字典']=df0.apply(lambda x: 名值们toDict(x['名值们'],x) , axis = 1)

# df=df0.drop(columns=['名值们'])
df=df0 #不删除列 '名值们'


### 原来的

In [11]:
df0.info()
df0.head(8)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74162 entries, 0 to 74161
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   函数分类    74162 non-null  object
 1   函数名     74162 non-null  object
 2   此行在区    74162 non-null  object
 3   此行内容    74162 non-null  object
 4   名值们     74162 non-null  object
 5   名值字典    74162 non-null  object
dtypes: object(6)
memory usage: 3.4+ MB


,函数分类,函数名,此行在区,此行内容,名值们,名值字典
0,模拟指令LGDT,LGDT_Ms,此行在区,此行内容,"cpu_mode:0,_CS_selector_value:0xf000,_CS_selec...","{'cpu_mode': '0', '_CS_selector_value': '0xf00..."
1,模拟指令LGDT,LGDT_Ms,此行在区,此行内容,"cpu_mode:0,_CS_selector_value:0x9020,_CS_selec...","{'cpu_mode': '0', '_CS_selector_value': '0x902..."
2,模拟指令LGDT,LGDT_Ms,此行在区,此行内容,"cpu_mode:2,_CS_selector_value:0x8,_CS_selector...","{'cpu_mode': '2', '_CS_selector_value': '0x8',..."
3,模拟指令LTR,LTR_Ew,此行在区,此行内容,"cpu_mode:2,_CS_selector_value:0x8,_CS_selector...","{'cpu_mode': '2', '_CS_selector_value': '0x8',..."
4,模拟指令LLDT,LLDT_Ew,此行在区,此行内容,"cpu_mode:2,_CS_selector_value:0x8,_CS_selector...","{'cpu_mode': '2', '_CS_selector_value': '0x8',..."
5,未分类,protected_mode_int,此行在区gate_descriptor.type=14,此行内容load_cs、load_ss、EIP=gate_dest_offset,"gate_descriptor.type:14,EIPorigin:0x68d7,vecto...","{'gate_descriptor.type': '14', 'EIPorigin': '0..."
6,未分类,protected_mode_int,此行在区gate_descriptor.type=15,此行内容load_cs、load_ss、EIP=gate_dest_offset,"gate_descriptor.type:15,EIPorigin:0x68fa,vecto...","{'gate_descriptor.type': '15', 'EIPorigin': '0..."
7,未分类,protected_mode_int,此行在区gate_descriptor.type=15,此行内容load_cs、load_ss、EIP=gate_dest_offset,"gate_descriptor.type:15,EIPorigin:0x690c,vecto...","{'gate_descriptor.type': '15', 'EIPorigin': '0..."


### 转换后的

In [12]:
df.info()
df.head(8)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74162 entries, 0 to 74161
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   函数分类    74162 non-null  object
 1   函数名     74162 non-null  object
 2   此行在区    74162 non-null  object
 3   此行内容    74162 non-null  object
 4   名值们     74162 non-null  object
 5   名值字典    74162 non-null  object
dtypes: object(6)
memory usage: 3.4+ MB


,函数分类,函数名,此行在区,此行内容,名值们,名值字典
0,模拟指令LGDT,LGDT_Ms,此行在区,此行内容,"cpu_mode:0,_CS_selector_value:0xf000,_CS_selec...","{'cpu_mode': '0', '_CS_selector_value': '0xf00..."
1,模拟指令LGDT,LGDT_Ms,此行在区,此行内容,"cpu_mode:0,_CS_selector_value:0x9020,_CS_selec...","{'cpu_mode': '0', '_CS_selector_value': '0x902..."
2,模拟指令LGDT,LGDT_Ms,此行在区,此行内容,"cpu_mode:2,_CS_selector_value:0x8,_CS_selector...","{'cpu_mode': '2', '_CS_selector_value': '0x8',..."
3,模拟指令LTR,LTR_Ew,此行在区,此行内容,"cpu_mode:2,_CS_selector_value:0x8,_CS_selector...","{'cpu_mode': '2', '_CS_selector_value': '0x8',..."
4,模拟指令LLDT,LLDT_Ew,此行在区,此行内容,"cpu_mode:2,_CS_selector_value:0x8,_CS_selector...","{'cpu_mode': '2', '_CS_selector_value': '0x8',..."
5,未分类,protected_mode_int,此行在区gate_descriptor.type=14,此行内容load_cs、load_ss、EIP=gate_dest_offset,"gate_descriptor.type:14,EIPorigin:0x68d7,vecto...","{'gate_descriptor.type': '14', 'EIPorigin': '0..."
6,未分类,protected_mode_int,此行在区gate_descriptor.type=15,此行内容load_cs、load_ss、EIP=gate_dest_offset,"gate_descriptor.type:15,EIPorigin:0x68fa,vecto...","{'gate_descriptor.type': '15', 'EIPorigin': '0..."
7,未分类,protected_mode_int,此行在区gate_descriptor.type=15,此行内容load_cs、load_ss、EIP=gate_dest_offset,"gate_descriptor.type:15,EIPorigin:0x690c,vecto...","{'gate_descriptor.type': '15', 'EIPorigin': '0..."


## 有哪些函数

In [13]:
df.函数名.unique().tolist()

['LGDT_Ms', 'LTR_Ew', 'LLDT_Ew', 'protected_mode_int']

### 这些函数各被调用次数

In [14]:
df.groupby(['函数名']).size()

函数名
LGDT_Ms                   3
LLDT_Ew                   1
LTR_Ew                    1
protected_mode_int    74157
dtype: int64

## protected_mode_int && BX_386_INTERRUPT_GATE

[bochs/cpu/descriptor.h](https://gitcode.net/crk/bochs/-/blob/master/bochs/cpu/descriptor.h)
```cpp
//文件 https://gitcode.net/crk/bochs/-/blob/master/bochs/cpu/descriptor.
enum { //第113行
  BX_GATE_TYPE_NONE            = 0x0,
  BX_SYS_SEGMENT_AVAIL_286_TSS = 0x1,
  BX_SYS_SEGMENT_LDT           = 0x2,
  BX_SYS_SEGMENT_BUSY_286_TSS  = 0x3,
  BX_286_CALL_GATE             = 0x4,
  BX_TASK_GATE                 = 0x5,
  BX_286_INTERRUPT_GATE        = 0x6,
  BX_286_TRAP_GATE             = 0x7,
                              /* 0x8 reserved */
  BX_SYS_SEGMENT_AVAIL_386_TSS = 0x9,
                              /* 0xa reserved */
  BX_SYS_SEGMENT_BUSY_386_TSS  = 0xb, // 11
  BX_386_CALL_GATE             = 0xc, //12
                              /* 0xd reserved */
  BX_386_INTERRUPT_GATE        = 0xe, // 14
  BX_386_TRAP_GATE             = 0xf, //15
};

```

In [15]:
df_pmi_386ig=df[ (df.函数名=='protected_mode_int')  &  (df.此行在区=='此行在区gate_descriptor.type=14') ]
df_pmi_386ig

,函数分类,函数名,此行在区,此行内容,名值们,名值字典
5,未分类,protected_mode_int,此行在区gate_descriptor.type=14,此行内容load_cs、load_ss、EIP=gate_dest_offset,"gate_descriptor.type:14,EIPorigin:0x68d7,vecto...","{'gate_descriptor.type': '14', 'EIPorigin': '0..."
10,未分类,protected_mode_int,此行在区gate_descriptor.type=14,此行内容load_cs、load_ss、EIP=gate_dest_offset,"gate_descriptor.type:14,EIPorigin:0x141ca,vect...","{'gate_descriptor.type': '14', 'EIPorigin': '0..."
11,未分类,protected_mode_int,此行在区gate_descriptor.type=14,此行内容load_cs、load_ss、EIP=gate_dest_offset,"gate_descriptor.type:14,EIPorigin:0x141ca,vect...","{'gate_descriptor.type': '14', 'EIPorigin': '0..."
21,未分类,protected_mode_int,此行在区gate_descriptor.type=14,此行内容load_cs、load_ss、EIP=gate_dest_offset,"gate_descriptor.type:14,EIPorigin:0x6adf,vecto...","{'gate_descriptor.type': '14', 'EIPorigin': '0..."
27,未分类,protected_mode_int,此行在区gate_descriptor.type=14,此行内容load_cs、load_ss、EIP=gate_dest_offset,"gate_descriptor.type:14,EIPorigin:0x6a71,vecto...","{'gate_descriptor.type': '14', 'EIPorigin': '0..."
...,...,...,...,...,...,...
74042,未分类,protected_mode_int,此行在区gate_descriptor.type=14,此行内容load_cs、load_ss、EIP=gate_dest_offset,"gate_descriptor.type:14,EIPorigin:0x6af8,vecto...","{'gate_descriptor.type': '14', 'EIPorigin': '0..."
74072,未分类,protected_mode_int,此行在区gate_descriptor.type=14,此行内容load_cs、load_ss、EIP=gate_dest_offset,"gate_descriptor.type:14,EIPorigin:0x6bcc,vecto...","{'gate_descriptor.type': '14', 'EIPorigin': '0..."
74101,未分类,protected_mode_int,此行在区gate_descriptor.type=14,此行内容load_cs、load_ss、EIP=gate_dest_offset,"gate_descriptor.type:14,EIPorigin:0x6b7d,vecto...","{'gate_descriptor.type': '14', 'EIPorigin': '0..."
74130,未分类,protected_mode_int,此行在区gate_descriptor.type=14,此行内容load_cs、load_ss、EIP=gate_dest_offset,"gate_descriptor.type:14,EIPorigin:0x6beb,vecto...","{'gate_descriptor.type': '14', 'EIPorigin': '0..."


### df_pmi_386ig : Dict列 名值字典 展开 为 多列 

In [16]:

#参考: https://stackoverflow.com/questions/64872676/how-to-keep-the-index-of-my-pandas-dataframe-after-normalazation-json/64873302#64873302
#    
_名值字典_展开_df_pmi_386ig = pandas.DataFrame.from_dict(df_pmi_386ig['名值字典'].to_dict(), orient='index')
#df_pmi_386ig_exp=pandas.concat([ df_pmi_386tg , _名值字典_展开_df_pmi_386ig], axis=1)
df_pmi_386ig_exp=pandas.concat([ df_pmi_386ig[['函数分类', '函数名', '此行内容', '此行在区']] , _名值字典_展开_df_pmi_386ig], axis=1)

###  展开结果

In [17]:
df_pmi_386ig_exp

,函数分类,函数名,此行内容,此行在区,gate_descriptor.type,EIPorigin,vector,soft_int,push_error,error_code,cs_selector.index,cs_descriptor.u.gate.dest_selector,cs_descriptor.u.gate.dest_offset,cs_descriptor.type,EIP
5,未分类,protected_mode_int,此行内容load_cs、load_ss、EIP=gate_dest_offset,此行在区gate_descriptor.type=14,14,0x68d7,0x20,0x0,0x0,0x0,0x1,0x0,0x0,0xb,0x781c
10,未分类,protected_mode_int,此行内容load_cs、load_ss、EIP=gate_dest_offset,此行在区gate_descriptor.type=14,14,0x141ca,0x2e,0x0,0x0,0x0,0x1,0x0,0x0,0xb,0x787a
11,未分类,protected_mode_int,此行内容load_cs、load_ss、EIP=gate_dest_offset,此行在区gate_descriptor.type=14,14,0x141ca,0x2e,0x0,0x0,0x0,0x1,0x0,0x0,0xb,0x787a
21,未分类,protected_mode_int,此行内容load_cs、load_ss、EIP=gate_dest_offset,此行在区gate_descriptor.type=14,14,0x6adf,0x20,0x0,0x0,0x0,0x1,0x0,0x0,0xb,0x781c
27,未分类,protected_mode_int,此行内容load_cs、load_ss、EIP=gate_dest_offset,此行在区gate_descriptor.type=14,14,0x6a71,0x2e,0x0,0x0,0x0,0x1,0x0,0x0,0xb,0x787a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74042,未分类,protected_mode_int,此行内容load_cs、load_ss、EIP=gate_dest_offset,此行在区gate_descriptor.type=14,14,0x6af8,0x20,0x0,0x0,0x0,0x1,0x0,0x0,0xb,0x781c
74072,未分类,protected_mode_int,此行内容load_cs、load_ss、EIP=gate_dest_offset,此行在区gate_descriptor.type=14,14,0x6bcc,0x20,0x0,0x0,0x0,0x1,0x0,0x0,0xb,0x781c
74101,未分类,protected_mode_int,此行内容load_cs、load_ss、EIP=gate_dest_offset,此行在区gate_descriptor.type=14,14,0x6b7d,0x20,0x0,0x0,0x0,0x1,0x0,0x0,0xb,0x781c
74130,未分类,protected_mode_int,此行内容load_cs、load_ss、EIP=gate_dest_offset,此行在区gate_descriptor.type=14,14,0x6beb,0x20,0x0,0x0,0x0,0x1,0x0,0x0,0xb,0x781c


In [18]:
df_pmi_386ig_exp_grp_EIPorigin_sz=df_pmi_386ig_exp.groupby(['EIPorigin']).size()
len(df_pmi_386ig_exp_grp_EIPorigin_sz)
df_pmi_386ig_exp_grp_EIPorigin_sz.sort_values(ascending=False)

92

EIPorigin
0x14cd9    493
0x6c04     360
0x6af8     356
0x6a71     341
0x6b7d     322
0x6bcc     270
0x6ac0     267
0x6adf     221
0x6beb     215
0xd360      34
0x6b60      31
0x6aa1      30
0x6bad      26
0x12072     20
0x6b5a      13
0x6b2b      12
0xd3e1      12
0x12033     12
0x773c      10
0x6b50      10
0x6c37       9
0x690c       9
0x14be2      8
0x777a       8
0xc253       8
0x12030      8
0x7746       6
0x6a5f       5
0xdbb7       5
0xa2ea       4
0x7769       4
0x11ef4      4
0x6c5c       4
0xc780       4
0xc726       4
0xbf4e       4
0xbf27       4
0x77b7       2
0x1a3a7      2
0xddf3       2
0xdb19       2
0x9dbb       2
0xc3ba       2
0x141ca      2
0x9974       2
0x99d1       2
0xbfb7       2
0x15f76      2
0xdec0       1
0xa5a4       1
0xb910       1
0xe2f9       1
0x9d9d       1
0xde8d       1
0xba01       1
0xbd9b       1
0xbef1       1
0xdaea       1
0xc21a       1
0xc207       1
0xc030       1
0xbf0f       1
0x11da       1
0x9b4a       1
0x39c2f      1
0x14738      1


In [19]:
df_pmi_386ig_exp.groupby(['vector']).size()

vector
0x20    2609
0x2e     610
dtype: int64

In [20]:
df_pmi_386ig_exp.groupby(['soft_int']).size()

soft_int
0x0    3219
dtype: int64

In [21]:
df_pmi_386ig_exp.groupby(['push_error']).size()

push_error
0x0    3219
dtype: int64

In [22]:
df_pmi_386ig_exp.groupby(['error_code']).size()

error_code
0x0    3219
dtype: int64

In [23]:
df_pmi_386ig_exp.groupby(['cs_selector.index']).size()

cs_selector.index
0x1    3219
dtype: int64

In [24]:
df_pmi_386ig_exp.groupby(['cs_descriptor.u.gate.dest_selector']).size()

cs_descriptor.u.gate.dest_selector
0x0    3219
dtype: int64

In [25]:
df_pmi_386ig_exp.groupby(['cs_descriptor.u.gate.dest_offset']).size()

cs_descriptor.u.gate.dest_offset
0x0    3219
dtype: int64

In [26]:
df_pmi_386ig_exp.groupby(['cs_descriptor.type']).size()

cs_descriptor.type
0xb    3219
dtype: int64

In [27]:
df_pmi_386ig_exp.groupby(['EIP']).size()

EIP
0x781c    2609
0x787a     610
dtype: int64

## protected_mode_int && BX_386_TRAP_GATE

In [28]:
df_pmi_386tg=df[ (df.函数名=='protected_mode_int')  &  (df.此行在区=='此行在区gate_descriptor.type=15') ]
# df_pmi_386tg

### df_pmi_386tg : Dict列 名值字典 展开 为 多列 

In [29]:
#参考: https://stackoverflow.com/questions/64872676/how-to-keep-the-index-of-my-pandas-dataframe-after-normalazation-json/64873302#64873302
#    
_名值字典_展开_df_pmi_386tg = pandas.DataFrame.from_dict(df_pmi_386tg['名值字典'].to_dict(), orient='index')
#df_pmi_386tg_exp=pandas.concat([ df_pmi_386tg , _名值字典_展开_df_pmi_386tg], axis=1)
df_pmi_386tg_exp=pandas.concat([ df_pmi_386tg[['函数分类', '函数名', '此行内容', '此行在区']] , _名值字典_展开_df_pmi_386tg], axis=1)

###  展开结果

In [30]:
df_pmi_386tg_exp

,函数分类,函数名,此行内容,此行在区,gate_descriptor.type,EIPorigin,vector,soft_int,push_error,error_code,cs_selector.index,cs_descriptor.u.gate.dest_selector,cs_descriptor.u.gate.dest_offset,cs_descriptor.type,EIP
6,未分类,protected_mode_int,此行内容load_cs、load_ss、EIP=gate_dest_offset,此行在区gate_descriptor.type=15,15,0x68fa,0x80,0x1,0x0,0x0,0x1,0x0,0x0,0xb,0x7746
7,未分类,protected_mode_int,此行内容load_cs、load_ss、EIP=gate_dest_offset,此行在区gate_descriptor.type=15,15,0x690c,0x80,0x1,0x0,0x0,0x1,0x0,0x0,0xb,0x7746
8,未分类,protected_mode_int,此行内容load_cs、load_ss、EIP=gate_dest_offset,此行在区gate_descriptor.type=15,15,0x693d,0xe,0x0,0x1,0x7,0x1,0x0,0x0,0xb,0xa5a4
9,未分类,protected_mode_int,此行内容load_cs、load_ss、EIP=gate_dest_offset,此行在区gate_descriptor.type=15,15,0x64f8,0x80,0x1,0x0,0x0,0x1,0x0,0x0,0xb,0x7746
12,未分类,protected_mode_int,此行内容load_cs、load_ss、EIP=gate_dest_offset,此行在区gate_descriptor.type=15,15,0x690c,0x80,0x1,0x0,0x0,0x1,0x0,0x0,0xb,0x7746
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74156,未分类,protected_mode_int,此行内容load_cs、load_ss、EIP=gate_dest_offset,此行在区gate_descriptor.type=15,15,0x690c,0x80,0x1,0x0,0x0,0x1,0x0,0x0,0xb,0x7746
74157,未分类,protected_mode_int,此行内容load_cs、load_ss、EIP=gate_dest_offset,此行在区gate_descriptor.type=15,15,0x690c,0x80,0x1,0x0,0x0,0x1,0x0,0x0,0xb,0x7746
74158,未分类,protected_mode_int,此行内容load_cs、load_ss、EIP=gate_dest_offset,此行在区gate_descriptor.type=15,15,0x690c,0x80,0x1,0x0,0x0,0x1,0x0,0x0,0xb,0x7746
74160,未分类,protected_mode_int,此行内容load_cs、load_ss、EIP=gate_dest_offset,此行在区gate_descriptor.type=15,15,0x690c,0x80,0x1,0x0,0x0,0x1,0x0,0x0,0xb,0x7746


In [31]:
df_pmi_386tg_exp_grp_EIPorigin_sz=df_pmi_386tg_exp.groupby(['EIPorigin']).size()
len(df_pmi_386tg_exp_grp_EIPorigin_sz)
df_pmi_386tg_exp_grp_EIPorigin_sz.sort_values(ascending=False)

164

EIPorigin
0x690c     69962
0x3d8e1      177
0x397bc      130
0x39a44      100
0x39800       55
0x3d912       30
0x1a29a       25
0x39b78       24
0x39a68       23
0x39988       22
0x399fc       22
0xd4          20
0x1a419       14
0x39894       12
0x1a432       11
0x397e0        9
0x1093c        8
0x16020        7
0x1a3bc        6
0x1a580        6
0x9a7a         6
0x398d0        6
0x39844        6
0x6adf         5
0x39a88        5
0x4            4
0x39903        4
0x39963        4
0x5fe9         4
0x51bc         4
0x0            4
0x398af        4
0x1825a        4
0x1820e        4
0x19575        4
0x109d7        4
0x39b28        3
0x6b7d         3
0x181ec        3
0x39933        3
0x399d4        3
0x145          3
0x39b9a        3
0x3dd6c        3
0x3d859        3
0xf0fd         3
0x6ac0         2
0x15ee8        2
0x8128         2
0xa0f4         2
0x1a3b0        2
0x39ae0        2
0x39aac        2
0x14a4c        2
0x39a1c        2
0x5620         2
0x16038        2
0xb098         2
0x39

In [32]:
df_pmi_386tg_exp.groupby(['vector']).size()

vector
0x21       16
0x80    70672
0xe       250
dtype: int64

In [33]:
df_pmi_386tg_exp.groupby(['soft_int']).size()

soft_int
0x0      266
0x1    70672
dtype: int64

In [34]:
df_pmi_386tg_exp.groupby(['push_error']).size()

push_error
0x0    70688
0x1      250
dtype: int64

In [35]:
df_pmi_386tg_exp.groupby(['error_code']).size()

error_code
0x0    70688
0x4      106
0x6       48
0x7       96
dtype: int64

In [36]:
df_pmi_386tg_exp.groupby(['cs_selector.index']).size()

cs_selector.index
0x1    70938
dtype: int64

In [37]:
df_pmi_386tg_exp.groupby(['cs_descriptor.u.gate.dest_selector']).size()

cs_descriptor.u.gate.dest_selector
0x0    70938
dtype: int64

In [38]:
df_pmi_386tg_exp.groupby(['cs_descriptor.u.gate.dest_offset']).size()

cs_descriptor.u.gate.dest_offset
0x0    70938
dtype: int64

In [39]:
df_pmi_386tg_exp.groupby(['cs_descriptor.type']).size()

cs_descriptor.type
0xb    70938
dtype: int64

In [40]:
df_pmi_386tg_exp.groupby(['EIP']).size()

EIP
0x173fe       16
0x7746     70672
0xa5a4       250
dtype: int64

# 开发用

In [41]:
_名值字典_展开_df_pmi_386ig

,gate_descriptor.type,EIPorigin,vector,soft_int,push_error,error_code,cs_selector.index,cs_descriptor.u.gate.dest_selector,cs_descriptor.u.gate.dest_offset,cs_descriptor.type,EIP
5,14,0x68d7,0x20,0x0,0x0,0x0,0x1,0x0,0x0,0xb,0x781c
10,14,0x141ca,0x2e,0x0,0x0,0x0,0x1,0x0,0x0,0xb,0x787a
11,14,0x141ca,0x2e,0x0,0x0,0x0,0x1,0x0,0x0,0xb,0x787a
21,14,0x6adf,0x20,0x0,0x0,0x0,0x1,0x0,0x0,0xb,0x781c
27,14,0x6a71,0x2e,0x0,0x0,0x0,0x1,0x0,0x0,0xb,0x787a
...,...,...,...,...,...,...,...,...,...,...,...
74042,14,0x6af8,0x20,0x0,0x0,0x0,0x1,0x0,0x0,0xb,0x781c
74072,14,0x6bcc,0x20,0x0,0x0,0x0,0x1,0x0,0x0,0xb,0x781c
74101,14,0x6b7d,0x20,0x0,0x0,0x0,0x1,0x0,0x0,0xb,0x781c
74130,14,0x6beb,0x20,0x0,0x0,0x0,0x1,0x0,0x0,0xb,0x781c


In [42]:
_名值字典_展开_df_pmi_386tg

,gate_descriptor.type,EIPorigin,vector,soft_int,push_error,error_code,cs_selector.index,cs_descriptor.u.gate.dest_selector,cs_descriptor.u.gate.dest_offset,cs_descriptor.type,EIP
6,15,0x68fa,0x80,0x1,0x0,0x0,0x1,0x0,0x0,0xb,0x7746
7,15,0x690c,0x80,0x1,0x0,0x0,0x1,0x0,0x0,0xb,0x7746
8,15,0x693d,0xe,0x0,0x1,0x7,0x1,0x0,0x0,0xb,0xa5a4
9,15,0x64f8,0x80,0x1,0x0,0x0,0x1,0x0,0x0,0xb,0x7746
12,15,0x690c,0x80,0x1,0x0,0x0,0x1,0x0,0x0,0xb,0x7746
...,...,...,...,...,...,...,...,...,...,...,...
74156,15,0x690c,0x80,0x1,0x0,0x0,0x1,0x0,0x0,0xb,0x7746
74157,15,0x690c,0x80,0x1,0x0,0x0,0x1,0x0,0x0,0xb,0x7746
74158,15,0x690c,0x80,0x1,0x0,0x0,0x1,0x0,0x0,0xb,0x7746
74160,15,0x690c,0x80,0x1,0x0,0x0,0x1,0x0,0x0,0xb,0x7746
